Sample image classification

In [1]:
%%capture
!pip3 install geemap

In [2]:
import ee
import geemap
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='ee-rajaoberison')

In [3]:
# SCALE OF THE STUDY
scale = 30; # meters
Map = geemap.Map()

In [4]:
# FUNCTIONS
## OTSU
# https://medium.com/google-earth/otsus-method-for-image-segmentation-f5c48f405e
def otsu(histogram):
  counts = ee.Array(ee.Dictionary(histogram).get('histogram'));
  means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'));
  size = means.length().get([0]);
  total = counts.reduce(ee.Reducer.sum(), [0]).get([0]);
  sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0]);
  mean = sum.divide(total);

  indices = ee.List.sequence(1, size);

  # Compute between sum of squares, where each mean partitions the data.
  def ssq(i):
    aCounts = counts.slice(0, 0, i);
    aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0]);
    aMeans = means.slice(0, 0, i);
    aMean = aMeans.multiply(aCounts)\
        .reduce(ee.Reducer.sum(), [0]).get([0])\
        .divide(aCount);
    bCount = total.subtract(aCount);
    bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount);
    return aCount.multiply(aMean.subtract(mean).pow(2)).add(
            bCount.multiply(bMean.subtract(mean).pow(2)));
  bss = indices.map(ssq);

  # Return the mean value corresponding to the maximum BSS.
  return means.sort(bss).get([-1]);

## RANDOM FOREST CLASSIFIER
def RFclassifier(image, training0, training1, trainingbands, scale):
  training0 = training0.geometry(0.1);
  training1 = training1.geometry(0.1);

  # IMAGE CLASSIFICATION
  # Create random points inside polygons for training and validation
  tpts0 = ee.FeatureCollection.randomPoints(training0, 2000, 0);
  not_tpts0 = ee.FeatureCollection.randomPoints(training1, 2000, 0);

  # Take a random sample inside the polygons for validation
  vpts = ee.FeatureCollection.randomPoints(training0, 600, 1);
  not_vpts = ee.FeatureCollection.randomPoints(training1, 600, 1);

  # ADD CLASS FIELD
  def addField(training0):
    return training0.set({'landcover': 1});
  def addField2(training1):
    return training1.set({'landcover': 0});

  tpts = tpts0.map(addField);
  not_tpts = not_tpts0.map(addField2);
  vpts = vpts.map(addField);
  not_vpts = not_vpts.map(addField2);

  # Merging random points
  trainingpts = tpts.merge(not_tpts);
  validpts = vpts.merge(not_vpts);

  # Train the classifier
  # Sample the input imagery to get a FeatureCollection of training data.
  training = image.sampleRegions(
    collection= trainingpts,
    properties= ['landcover'],
    scale= scale
  );
  validation = image.sampleRegions(
    collection= validpts,
    properties= ['landcover'],
    scale= scale
  );

  # Make a random forest classifier and train it.
  classifier = ee.Classifier.smileRandomForest(10)\
    .train(training, 'landcover', trainingbands);

  # Classify the input imagery.
  classified = image.select(trainingbands)\
    .classify(classifier).rename('class');
  vClassified = validation.classify(classifier);
  testAccuracy = vClassified.errorMatrix('landcover', 'classification');

  return {'t':classified, 'v':vClassified, 'a':testAccuracy};

## LANDSAT 5 IMAGE CLASSIFIER
def l5classifier(year, aoi, training_region, scale):

  # IMAGE PREPARATION
  # Get Landsat Images
  # Year is defined as the Tropical cyclone season
  # https://en.wikipedia.org/wiki/2018%E2%80%9319_South-West_Indian_Ocean_cyclone_season
  year_0 = year - 1;
  raw = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
      .filterDate(str(year_0)+'-05-01', str(year)+'-10-31').filterBounds(aoi)\
      .filter(ee.Filter.lte('CLOUD_COVER_LAND', 10));

  visImage = {'bands': ['SR_B4', 'SR_B5', 'SR_B1'],
              'min': 140, 'max': 4300};

  # CLOUD REMOVAL SCRIPT FROM GEE EXAMPLES
  # This example demonstrates the use of the Landsat 4, 5 or 7
  # surface reflectance QA band to mask clouds.
  def cloudMaskL457(image):
    qa = image.select('QA_PIXEL');
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5)\
      .And(qa.bitwiseAnd(1 << 7))\
      .Or(qa.bitwiseAnd(1 << 3));
    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min());
    return image.updateMask(cloud.Not()).updateMask(mask2)

  # Map the function over the collection, take the median, and clip.

  cloudRemoved = raw\
      .map(cloudMaskL457)\
      .median();

  # REMOVING WATER
  # OTSU THRESHOLDING TECHNIQUE
  # Also water classification: water vs. non-water

  # Compute the histogram of the NIR band.  The mean and variance are only FYI.
  histogram = cloudRemoved.select('SR_B4').reduceRegion(
    reducer= ee.Reducer.histogram(255, 2)
        .combine('mean', None, True)
        .combine('variance', None, True),
    geometry= aoi,
    scale= scale,
    bestEffort= False
  );

  # Chart the histogram
  threshold = otsu(histogram.get('SR_B4_histogram'));

  waterMask = cloudRemoved.select('SR_B4').gt(threshold);

  waterMasked = cloudRemoved.mask(waterMask);

  # SUBSETTING BY INTERTIDAL / MANGROVE AREAS
  # IMPORT GIRI (2011) AS REFERENCE
  giri = ee.ImageCollection('LANDSAT/MANGROVE_FORESTS').filterBounds(aoi);

  # BASED ON DEM VARIANCE
  dem = ee.Image('JAXA/ALOS/AW3D30_V1_1').clip(aoi).select('AVE');
  demPalette = ['blue', 'lightBlue', 'darkGreen', 'brown', 'white'];

  # Not suitable for mangroves, if elevation below 30m
  # below30m = dem.lte(30);

  # Actual Interdidal zones using tide data
  # https://www.tideschart.com/Madagascar/Diana/Nosy-Be/
  intertidal = cloudRemoved.updateMask(
      ee.ImageCollection([giri.mosaic().focal_mode(10).toInt(),
                          dem.mask(dem.lte(10)).rename('1').toInt()
                        ]).mosaic().updateMask(waterMask)
                        ).clip(aoi);

  # TRAINING DATA
  # Training polygons
  sand = training_region.filter(ee.Filter.eq('landcover', '0'));
  mangroves = training_region.filter(ee.Filter.eq('landcover', '1'));
  deg_mangroves = training_region.filter(ee.Filter.eq('landcover', '2'));
  agri = training_region.filter(ee.Filter.eq('landcover', '4'));

  notmangroves = sand.merge(deg_mangroves).merge(agri);
  notsand = deg_mangroves.merge(agri);
  terrestrial_veg = agri;

  # IMAGE ANALYSIS
  # MAPPING MANGROVES
  final = intertidal;
  # WATER AND VEGETATION INDEXES
  # NDVI
  ndvi = final.normalizedDifference(['SR_B4', 'SR_B3']).rename('ndvi');
  vegPalette = ['blue', 'white', 'darkgreen'];

  # EVI
  evi0 = final.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
      {
        'NIR': final.select('SR_B4'),
        'RED': final.select('SR_B3'),
        'BLUE': final.select('SR_B1')
      }
    );
  evi = evi0.select('constant').rename('evi');

  # Ref: Bo-cai Gao, 1996, NDWI—A normalized difference water index for remote sensing of vegetation
  # liquid water from space,Remote Sensing of Environment, Volume 58, Issue 3, Pages 257-266,
  # https://doi.org/10.1016/S0034-4257(96)00067-3.
  # http://www.sciencedirect.com/science/article/pii/S0034425796000673)
  # NDWI
  ndwi = final.normalizedDifference(['SR_B4', 'SR_B5']).rename('ndwi');

  # Ref: Hanqiu Xu (2006) Modification of normalised difference water index (NDWI)
  # to enhance open water features in remotely sensed imagery, International Journal of Remote
  # Sensing, 27:14, 3025-3033, DOI: 10.1080/01431160600589179
  # MNDWI
  mndwi = final.normalizedDifference(['SR_B2', 'SR_B5']).rename('mndwi');

  # Band ratios: reference Green, E.P.; Clark, C.D.; Mumby, P.J.; Edwards, A.J.;
  # Ellis, A.C. Remote sensing techniques for mangrove mapping. Int. J. Remote
  # Sens. 1998, 19, 935–956.

  # Band swir/nir ratio (band 5:4 for Landsat5, 6:5 for Landsat 8)
  ratio54 = final.select('SR_B5').divide(final.select('SR_B4')).rename('ratio54');

  # Band Red:SWIR ratio (band 3:5 for landsat 5, 4:6 for landsat 8)
  ratio35 = final.select('SR_B3').divide(final.select('SR_B5')).rename('ratio35');

  # Prep for classification: stack all bands, indicies, ratios
  final_stack = final.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(evi).addBands(ratio54).addBands(ratio35);

  trainingbands = ee.List(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7','ndvi', 'ndwi',
                                'mndwi','evi','ratio54','ratio35']);

  classified = RFclassifier(final_stack, mangroves, notmangroves, trainingbands, scale);
  mang_acc = classified['a'];

  # Extract Mangroves
  # Create a binary mask from classification
  mangrove_mask = ee.Image(classified['t']).select('class').eq(1);
  classified_mangrove = ee.Image(classified['t']).updateMask(mangrove_mask);

  # MAPPING TERRESTRIAL LAND
  notmangrove_mask = ee.Image(classified['t']).select('class').eq(0);
  notmangrove_zones = intertidal.updateMask(notmangrove_mask);

  # TASSELED CAP TRANSFORMATION
  # Define an Array of Tasseled Cap coefficients.
  coefficients = ee.Array([
    [0.3037, 0.2793, 0.4743, 0.5585, 0.5082, 0.1863],
    [-0.2848, -0.2435, -0.5436, 0.7243, 0.0840, -0.1800],
    [0.1509, 0.1973, 0.3279, 0.3406, -0.7112, -0.4572],
    [-0.8242, 0.0849, 0.4392, -0.0580, 0.2012, -0.2768],
    [-0.3280, 0.0549, 0.1075, 0.1855, -0.4357, 0.8085],
    [0.1084, -0.9022, 0.4120, 0.0573, -0.0251, 0.0238]
  ]);

  # Make an Array Image, with a 1-D Array per pixel.
  arrayImage1D = notmangrove_zones.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).toArray();

  # Make an Array Image with a 2-D Array per pixel, 6x1.
  arrayImage2D = arrayImage1D.toArray(1);

  # Do a matrix multiplication: 6x6 times 6x1.
  tasseled = ee.Image(coefficients)\
    .matrixMultiply(arrayImage2D)\
    .arrayProject([0])\
    .arrayFlatten(
      [['brightness', 'greenness', 'wetness', 'fourth', 'fifth', 'sixth']]);

  # Display the first three bands of the result and the input imagery.
  vizParams = {
    'bands': ['brightness', 'greenness', 'wetness'],
    'min': -0.1, 'max': [0.5, 0.1, 0.1]
  };

  # Map.addLayer(tasseled, vizParams, 'components');

  terr = notmangrove_zones.addBands(tasseled.select('brightness'))\
    .addBands(tasseled.select('greenness'))\
    .addBands(tasseled.select('wetness'));

  trainingbands_2 = ee.List([
      'SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7',
      'brightness', 'greenness','wetness']);

  classified_2 = RFclassifier(terr, sand, notsand, trainingbands_2, scale);

  sand_mask = ee.Image(classified_2['t']).select('class').eq(1);
  classified_sand = ee.Image(classified_2['t']).updateMask(sand_mask);

  # MAPPING TERRESTRIAL VEGETATION
  # We don't have multiseries options so we'll use indexes
  # https://medium.com/regen-network/remote-sensing-indices-389153e3d947
  green_mask = ee.Image(classified_2['t']).select('class').eq(0);
  green_zones = intertidal.updateMask(green_mask);

  avi = green_zones.expression(
      'cbrt((B4 + 1) * (256 - B3) * (B4 - B3))',
      {
        'B4': green_zones.select('SR_B4'),
        'B3': green_zones.select('SR_B3')
      }
    );
  avi = avi.rename('avi');

  bi = green_zones.expression(
      '((B4 + B2) - B3)/((B4 + B2) + B3)',
      {
        'B4': green_zones.select('SR_B4'),
        'B3': green_zones.select('SR_B3'),
        'B2': green_zones.select('SR_B2')
      }
    );
  bi = bi.rename('bi');

  si = green_zones.expression(
      'sqrt((256 - B2) * (256 - B3))',
      {
        'B2': green_zones.select('SR_B2'),
        'B3': green_zones.select('SR_B3')
      }
    );
  si = si.rename('si');

  terr_forest = green_zones.addBands(avi).addBands(bi).addBands(si);

  trainingbands_3 = ee.List(['avi', 'bi', 'si']);

  classified_3 = RFclassifier(terr_forest, deg_mangroves, terrestrial_veg, trainingbands_3, scale);

  deg_mask = ee.Image(classified_3['t']).select('class').eq(1);
  classified_deg = ee.Image(classified_3['t']).updateMask(deg_mask);

  # MAPPING AGRI
  green2_mask = ee.Image(classified_3['t']).select('class').eq(0);
  classified_agri = ee.Image(classified_3['t']).updateMask(green2_mask);

  all = ee.ImageCollection.fromImages([
    classified_mangrove.select('class').rename(str(year)).multiply(5).toInt(),
    classified_sand.select('class').rename(str(year)).multiply(1).toInt(),
    classified_deg.select('class').rename(str(year)).multiply(4).toInt(),
    classified_agri.select('class').rename(str(year)).add(2).toInt()
    ]);

  return {'image':all.mosaic(), 'accuracy':mang_acc};



In [5]:
# MAIN CODE
## Area of interest
belo = ee.Geometry.Polygon(
        [[[45.60745486352051, -15.70059242683455],
          [45.60745486352051, -15.984634363884235],
          [46.177370635004884, -15.984634363884235],
          [46.177370635004884, -15.70059242683455]]]);
Map.centerObject(belo, 11);
Map.addLayer(belo, {}, 'AOI', False);
training = ee.FeatureCollection("users/rajaoberison/mg_mangroves");

## LAND COVER CLASSIFICATION GIVEN THE REGION OF STUDY
# From 2000 to 2008 with a four-year intervall
cover_2000 = l5classifier(2000, belo, training, scale);
cover_2004 = l5classifier(2004, belo, training, scale);
cover_2008 = l5classifier(2008, belo, training, scale);

# outputs
classesViz = {min:0, max:4,
  'palette': ['FFFFCC','CCFF00','9999FF','FF33FF']};
Map.addLayer(cover_2000['image'], classesViz, '2000', False);
Map.addLayer(cover_2004['image'], classesViz, '2004', False);
Map.addLayer(cover_2008['image'], classesViz, '2008', True);

In [6]:
accuracy = cover_2008['accuracy'].accuracy()
accuracy

In [7]:
Map

Map(center=[-15.842766195521678, 45.89241274926292], controls=(WidgetControl(options=['position', 'transparent…